## Similaridade Entre Produtos

Um script simples que permite a análise de similaridade entre 2 produtos diferentes utilizando geometria analítica - Similaridade/Distância do Cosseno. Para esta implementação, calculamo a distância angular de dois vetores compostos dos tokens gerados das strings de nossos produtos.


![Esquema](https://www.researchgate.net/profile/Claudio-Lima-15/publication/370107074/figure/fig4/AS:11431281151632568@1681994414598/Figura-12-Medida-cosseno-de-similaridade-de-documentos.png)

#### 1 - Importação e Definição da função de Similaridade

In [46]:
import pandas as pd
import math
import itertools
import time

#### 2 - Modelagem da Similaridade/Distância do Cosseno

##### 2.1 - Definição das Strings

In [47]:
str1 = "Zapatillas Nike"
str2 = "Zapatillas Adidas"

##### 2.2 - Tokenização com conjunto único

In [48]:
tokens_str1 = str1.split()
tokens_str2 = str2.split()

unique_tokens = set(tokens_str1 + tokens_str2)
unique_tokens

{'Adidas', 'Nike', 'Zapatillas'}

##### 2.3 - Vetorização

In [49]:
vec_str1 = [tokens_str1.count(token) for token in unique_tokens]
vec_str2 = [tokens_str2.count(token) for token in unique_tokens]

print(vec_str1)
print(vec_str2)

[1, 0, 1]
[1, 1, 0]


##### 2.4 - Cálculo do Produto Interno dos Vetores

In [50]:
dot_product = sum(i*j for i, j in zip(vec_str1, vec_str2))
dot_product

1

##### 2.5 - Cálculo da Magnitude dos vetores

In [51]:
magnitude_str1 = math.sqrt(sum(i**2 for i in vec_str1))
magnitude_str2 = math.sqrt(sum(i**2 for i in vec_str2))
print(magnitude_str1)
print(magnitude_str2)

1.4142135623730951
1.4142135623730951


##### 2.6 - Cálculo da Similaridade do Cosseno

Calcula-se a similaridade através da fórmula abaixo (Produto interno / Magnitude de cada Vetor), obtendo o mesmo valor encontrado no enunciado do case

![Fórmula](https://miro.medium.com/v2/resize:fit:858/0*7dWSG0979AvY4Wo2)

In [52]:
similarity = dot_product / (magnitude_str1 * magnitude_str2)
round(similarity,2)

0.5

#### 3 - Aplicação do Script

##### 3.1 - Definição da Função de Similaridade

In [53]:
def cosine_similarity(row):
    str1 = row['Item 1']
    str2 = row['Item 2']

    tokens_str1 = str1.split()
    tokens_str2 = str2.split()
    
    unique_tokens = set(tokens_str1 + tokens_str2)
    
    vec_str1 = [tokens_str1.count(token) for token in unique_tokens]
    vec_str2 = [tokens_str2.count(token) for token in unique_tokens]
    
    dot_product = sum(i*j for i, j in zip(vec_str1, vec_str2))
    
    magnitude_str1 = math.sqrt(sum(i**2 for i in vec_str1))
    magnitude_str2 = math.sqrt(sum(i**2 for i in vec_str2))
    
    similarity = dot_product / (magnitude_str1 * magnitude_str2)
    
    return similarity

##### 3.2 - Carregamento dos Dados

In [54]:
df = pd.read_csv('data/items_titles_test.csv')
df.head(5)

,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde


##### 3.3 - Verificação dos Dados

In [55]:
print(len(df.loc[df.duplicated()]))


0


In [56]:
print(df.isna().sum())

ITE_ITEM_TITLE    0
dtype: int64


##### 3.4 - Modelagem e Adaptação para implementação do Script

Nesta etapa, utilizamos um exemplo de produto para ser comparado com os outros itens da lista

In [57]:
product = 'Tênis Olympikus Esporte Valente - Masculino Kids'
product_list = df['ITE_ITEM_TITLE'].tolist()

combinations = list(itertools.product([product], product_list))
df_report = pd.DataFrame(combinations, columns=['Item 1', 'Item 2'])
df_report.head(5)

,Item 1,Item 2
0,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Olympikus Esporte Valente - Masculino Kids
1,Tênis Olympikus Esporte Valente - Masculino Kids,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Star Baby Sapatinho Conforto + Brinde


É necessário a remoção do mesmo produto entre ambos os itens

In [58]:
df_report= df_report.loc[df_report['Item 1'] != df_report['Item 2']]
df_report.head(5)

,Item 1,Item 2
1,Tênis Olympikus Esporte Valente - Masculino Kids,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Star Baby Sapatinho Conforto + Brinde
5,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Oakley Frequency 3.0 Preto/marrom


##### 3.5 - Aplicação da Função de Similaridade

Aplica-se a função para comparar cada item da lista, ordenando os maiores scores de similaridade e obtendo o tempo de execução

In [59]:
t1 = time.time()
df_report['Score Similitud (0,1)'] = df_report.apply(cosine_similarity, axis=1)
df_report = df_report.sort_values(by='Score Similitud (0,1)', ascending=False)
print(f'{round(time.time() - t1, 2)}s de execução para 10k produtos')
df_report

0.46s de execução para 10k produtos


,Item 1,Item 2,"Score Similitud (0,1)"
4241,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Olympikus Esporte Valente - Masculino In...,0.801784
7922,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Olympikus Masculino Valente - Kids Infan...,0.755929
642,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Olympikus Perfil - Masculino,0.676123
3271,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Olympikus Fourty - Masculino,0.676123
6977,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Olympikus Masculino - Preto/verde,0.676123
...,...,...,...
7686,Tênis Olympikus Esporte Valente - Masculino Kids,Reebok Atv 19,0.000000
3504,Tênis Olympikus Esporte Valente - Masculino Kids,Tenis Feminino Casual Maryland,0.000000
7690,Tênis Olympikus Esporte Valente - Masculino Kids,Tenis Infantil Feminino Luzinha De Led Que Pis...,0.000000
7691,Tênis Olympikus Esporte Valente - Masculino Kids,Bicicleta 29 Totem Pro Alivio Com 3x 9v Susp T...,0.000000


Observamos assim, que para a comparação de um produto com outros 10 mil em uma lista, temos o tempo de execução de aproximadamente **0.49 segundos**.

Novos produtos podem ser verificados e comparados sem a necessidade de treinamento de um modelo, justificando assim o emprego deste script e descatando a sua**escalabilidade**.

#### 4 - Exportação dos Dados

In [60]:
df_report.to_excel('output/similaridades.xlsx', index=False)